In [ ]:
# written by Maximilian Ullherr, maximilian.ullherr@physik.uni-wuerzburg.de,
# Lehrstuhl fuer Roentgenmikroskopie/Universitaet Wuerzburg, Josef-Martin-Weg 63, 97074 Wuerzburg, Germany
# last updated 2019-02-26

%pylab
import sys; sys.path.append('..')  # copy the pyETX module either into the folder with the notebook and remove this line
from SNR_u_evaluation import *

print('\n=== printing the docstring for the main function ===')
help(estimate_SNR)

# Template

These cells are intended to be modified to run an evaluation on actual data.

In [ ]:
# a) define data folders

# fill folder names here:
base_path = ''
images_folders = [os.path.join(base_path, folder) for folder in [
    'images1',
    'images2'
]]
refs_folders = [os.path.join(base_path, folder) for folder in [
    'refs1', # entries may be None
    'refs2'
]]
dark_folders = [os.path.join(base_path, folder) for folder in [
    'darks1', # entries may be None
    'darks2'
]]

view3 = slice(None, None), slice(None, None), slice(None, None)  # default value for slice

# load stack average to select evaluation area
from file_access import ImagesReader, save_image
image = ImagesReader(images_folders[0]).get_stack_average()
ref = ImagesReader(refs_folders[0]).get_stack_average()
dark = ImagesReader(dark_folders[0]).get_stack_average()

test_image = (image - dark)/(ref - dark)
fig1 = show_image(test_image)
display(fig1)

In [ ]:
# b) select evaluation area
# select an area that contains lots of signal
# all compared parameters need to be evaluated at the same area or need to be measured on a homogenous sample
view = slice(300, 800), slice(200, 700)
fig2 = show_image(test_image[view])
display(fig2)
view3 = slice(None, None), *view

In [ ]:
# c) compute and plot the SNR curves
# fill parameters here
result_path = ''
description_str = ''
parameter_1 = []  # use only if needed
parameter_2 = []  # use only if needed

# refs = ImagesReader(0, refs_folder).load_all()  # use this line if all series use the same refs 
                                                  # define refs_folder above

filterer = ImageSeriesPixelArtifactFilterer()

results = []
figure = None
for k in range(len(images_folders)):
    data = ImagesReader(images_folders[k]).load_all()
    refs = ImagesReader(refs_folders[k]).load_all() # possibly move out of the loop if only one ref
    darks = ImagesReader(dark_folders[k]).load_all() # if not needed, delete this line and usage below
    
    SNR_eval = SNR_Evaluator()
     
    SNR_eval.estimate_SNR(data[view3], refs[view3], darks[view3], series_filterer=filterer,
        save_path=os.path.join(result_path, f'SNR(u)_{description_str}_{parameter_1[k]:.2f}_{parameter_2[k]:.2f}'))
    
    figure = SNR_eval.plot(figure, f'{description_str}_{parameter_1[k]:.2f}_{parameter_2[k]:.2f}')
    # passing an existing figure here will plot all SNR curves into one figure
    
    results.append(SNR_eval)

SNR_eval.finalize_figure(figure, save_path=None) # this saves the plots
display(figure)